### Importing Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  46.33 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  35.91 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  39.82 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  45.94 MB/s
Libraries imported.


### Transforming Wikitable into Pandas Dataframe and Cleaning Up

In [2]:
# Copying and Pasting Wikitable
html_string = """"
<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Lawrence_Manor" title="Lawrence Manor">Lawrence Manor</a>
</td></tr>
<tr>
<td>M7A</td>
<td><a href="/wiki/Queen%27s_Park_(Toronto)" title="Queen&#39;s Park (Toronto)">Queen's Park</a></td>
<td>Not assigned
</td></tr>
<tr>
<td>M8A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Islington_Avenue" class="mw-redirect" title="Islington Avenue">Islington Avenue</a>
</td></tr>
<tr>
<td>M1B</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Rouge,_Toronto" title="Rouge, Toronto">Rouge</a>
</td></tr>
<tr>
<td>M1B</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Malvern,_Toronto" title="Malvern, Toronto">Malvern</a>
</td></tr>
<tr>
<td>M2B</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td>Don Mills North
</td></tr>
<tr>
<td>M4B</td>
<td><a href="/wiki/East_York" title="East York">East York</a></td>
<td><a href="/wiki/Woodbine_Gardens" class="mw-redirect" title="Woodbine Gardens">Woodbine Gardens</a>
</td></tr>
<tr>
<td>M4B</td>
<td><a href="/wiki/East_York" title="East York">East York</a></td>
<td><a href="/wiki/Parkview_Hill" class="mw-redirect" title="Parkview Hill">Parkview Hill</a>
</td></tr>
<tr>
<td>M5B</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Ryerson" title="Ryerson">Ryerson</a>
</td></tr>
<tr>
<td>M5B</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td>Garden District
</td></tr>
<tr>
<td>M6B</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Glencairn,_Ontario" class="mw-redirect" title="Glencairn, Ontario">Glencairn</a>
</td></tr>
<tr>
<td>M7B</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8B</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9B</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Cloverdale
</td></tr>
<tr>
<td>M9B</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Islington" title="Islington">Islington</a>
</td></tr>
<tr>
<td>M9B</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Martin Grove
</td></tr>
<tr>
<td>M9B</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Princess_Gardens" title="Princess Gardens">Princess Gardens</a>
</td></tr>
<tr>
<td>M9B</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/West_Deane_Park" class="mw-redirect" title="West Deane Park">West Deane Park</a>
</td></tr>
<tr>
<td>M1C</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Highland_Creek_(Toronto)" title="Highland Creek (Toronto)">Highland Creek</a>
</td></tr>
<tr>
<td>M1C</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Rouge_Hill" class="mw-redirect" title="Rouge Hill">Rouge Hill</a>
</td></tr>
<tr>
<td>M1C</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Port_Union,_Toronto" title="Port Union, Toronto">Port Union</a>
</td></tr>
<tr>
<td>M2C</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3C</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Flemingdon_Park" title="Flemingdon Park">Flemingdon Park</a>
</td></tr>
<tr>
<td>M3C</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td>Don Mills South
</td></tr>
<tr>
<td>M4C</td>
<td><a href="/wiki/East_York" title="East York">East York</a></td>
<td><a href="/wiki/Woodbine_Heights" class="mw-redirect" title="Woodbine Heights">Woodbine Heights</a>
</td></tr>
<tr>
<td>M5C</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/St._James_Town" title="St. James Town">St. James Town</a>
</td></tr>
<tr>
<td>M6C</td>
<td>York</td>
<td><a href="/wiki/Humewood-Cedarvale" class="mw-redirect" title="Humewood-Cedarvale">Humewood-Cedarvale</a>
</td></tr>
<tr>
<td>M7C</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8C</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9C</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Bloordale Gardens
</td></tr>
<tr>
<td>M9C</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Eringate
</td></tr>
<tr>
<td>M9C</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Markland_Wood" title="Markland Wood">Markland Wood</a>
</td></tr>
<tr>
<td>M9C</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Old Burnhamthorpe
</td></tr>
<tr>
<td>M1E</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td>Guildwood
</td></tr>
<tr>
<td>M1E</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Morningside,_Toronto" title="Morningside, Toronto">Morningside</a>
</td></tr>
<tr>
<td>M1E</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/West_Hill,_Toronto" title="West Hill, Toronto">West Hill</a>
</td></tr>
<tr>
<td>M2E</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3E</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M4E</td>
<td><a href="/wiki/East_Toronto" title="East Toronto">East Toronto</a></td>
<td><a href="/wiki/The_Beaches" title="The Beaches">The Beaches</a>
</td></tr>
<tr>
<td>M5E</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Berczy_Park" title="Berczy Park">Berczy Park</a>
</td></tr>
<tr>
<td>M6E</td>
<td>York</td>
<td>Caledonia-Fairbanks
</td></tr>
<tr>
<td>M7E</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8E</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9E</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M1G</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Woburn,_Toronto" title="Woburn, Toronto">Woburn</a>
</td></tr>
<tr>
<td>M2G</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3G</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M4G</td>
<td><a href="/wiki/East_York" title="East York">East York</a></td>
<td><a href="/wiki/Leaside" title="Leaside">Leaside</a>
</td></tr>
<tr>
<td>M5G</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td>Central Bay Street
</td></tr>
<tr>
<td>M6G</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td>Christie
</td></tr>
<tr>
<td>M7G</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8G</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9G</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M1H</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Woburn,_Toronto" title="Woburn, Toronto">Cedarbrae</a>
</td></tr>
<tr>
<td>M2H</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Hillcrest_Village" title="Hillcrest Village">Hillcrest Village</a>
</td></tr>
<tr>
<td>M3H</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Bathurst_Manor" title="Bathurst Manor">Bathurst Manor</a>
</td></tr>
<tr>
<td>M3H</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td>Downsview North
</td></tr>
<tr>
<td>M3H</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Wilson_Heights,_Toronto" class="mw-redirect" title="Wilson Heights, Toronto">Wilson Heights</a>
</td></tr>
<tr>
<td>M4H</td>
<td><a href="/wiki/East_York" title="East York">East York</a></td>
<td><a href="/wiki/Thorncliffe_Park" title="Thorncliffe Park">Thorncliffe Park</a>
</td></tr>
<tr>
<td>M5H</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Adelaide" title="Adelaide">Adelaide</a>
</td></tr>
<tr>
<td>M5H</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/King" title="King">King</a>
</td></tr>
<tr>
<td>M5H</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td>Richmond
</td></tr>
<tr>
<td>M6H</td>
<td><a href="/wiki/West_Toronto" title="West Toronto">West Toronto</a></td>
<td><a href="/wiki/Dovercourt_Village" class="mw-redirect" title="Dovercourt Village">Dovercourt Village</a>
</td></tr>
<tr>
<td>M6H</td>
<td><a href="/wiki/West_Toronto" title="West Toronto">West Toronto</a></td>
<td>Dufferin
</td></tr>
<tr>
<td>M7H</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8H</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9H</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M1J</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Scarborough_Village" title="Scarborough Village">Scarborough Village</a>
</td></tr>
<tr>
<td>M2J</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td>Fairview
</td></tr>
<tr>
<td>M2J</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Henry_Farm" title="Henry Farm">Henry Farm</a>
</td></tr>
<tr>
<td>M2J</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td>Oriole
</td></tr>
<tr>
<td>M3J</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Northwood_Park" class="mw-redirect" title="Northwood Park">Northwood Park</a>
</td></tr>
<tr>
<td>M3J</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/York_University" title="York University">York University</a>
</td></tr>
<tr>
<td>M4J</td>
<td><a href="/wiki/East_York" title="East York">East York</a></td>
<td><a href="/wiki/East_Toronto" title="East Toronto">East Toronto</a>
</td></tr>
<tr>
<td>M5J</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td>Harbourfront East
</td></tr>
<tr>
<td>M5J</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Toronto_Islands" title="Toronto Islands">Toronto Islands</a>
</td></tr>
<tr>
<td>M5J</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Union_Station_(Toronto)" title="Union Station (Toronto)">Union Station</a>
</td></tr>
<tr>
<td>M6J</td>
<td><a href="/wiki/West_Toronto" title="West Toronto">West Toronto</a></td>
<td><a href="/wiki/Little_Portugal,_Toronto" title="Little Portugal, Toronto">Little Portugal</a>
</td></tr>
<tr>
<td>M6J</td>
<td><a href="/wiki/West_Toronto" title="West Toronto">West Toronto</a></td>
<td><a href="/wiki/Trinity%E2%80%93Bellwoods" title="Trinity–Bellwoods">Trinity</a>
</td></tr>
<tr>
<td>M7J</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8J</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9J</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M1K</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td>East Birchmount Park
</td></tr>
<tr>
<td>M1K</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Ionview" title="Ionview">Ionview</a>
</td></tr>
<tr>
<td>M1K</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Kennedy_Park,_Toronto" class="mw-redirect" title="Kennedy Park, Toronto">Kennedy Park</a>
</td></tr>
<tr>
<td>M2K</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Bayview_Village" title="Bayview Village">Bayview Village</a>
</td></tr>
<tr>
<td>M3K</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/CFB_Toronto" title="CFB Toronto">CFB Toronto</a>
</td></tr>
<tr>
<td>M3K</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td>Downsview East
</td></tr>
<tr>
<td>M4K</td>
<td><a href="/wiki/East_Toronto" title="East Toronto">East Toronto</a></td>
<td>The Danforth West
</td></tr>
<tr>
<td>M4K</td>
<td><a href="/wiki/East_Toronto" title="East Toronto">East Toronto</a></td>
<td><a href="/wiki/Riverdale,_Toronto" title="Riverdale, Toronto">Riverdale</a>
</td></tr>
<tr>
<td>M5K</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Design_Exchange" title="Design Exchange">Design Exchange</a>
</td></tr>
<tr>
<td>M5K</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Toronto_Dominion_Centre" class="mw-redirect" title="Toronto Dominion Centre">Toronto Dominion Centre</a>
</td></tr>
<tr>
<td>M6K</td>
<td><a href="/wiki/West_Toronto" title="West Toronto">West Toronto</a></td>
<td>Brockton
</td></tr>
<tr>
<td>M6K</td>
<td><a href="/wiki/West_Toronto" title="West Toronto">West Toronto</a></td>
<td><a href="/wiki/Exhibition_Place" title="Exhibition Place">Exhibition Place</a>
</td></tr>
<tr>
<td>M6K</td>
<td><a href="/wiki/West_Toronto" title="West Toronto">West Toronto</a></td>
<td><a href="/wiki/Parkdale_Village" class="mw-redirect" title="Parkdale Village">Parkdale Village</a>
</td></tr>
<tr>
<td>M7K</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8K</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9K</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M1L</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Clairlea" title="Clairlea">Clairlea</a>
</td></tr>
<tr>
<td>M1L</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Golden_Mile,_Toronto" title="Golden Mile, Toronto">Golden Mile</a>
</td></tr>
<tr>
<td>M1L</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Oakridge,_Toronto" title="Oakridge, Toronto">Oakridge</a>
</td></tr>
<tr>
<td>M2L</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Silver_Hills" class="mw-redirect" title="Silver Hills">Silver Hills</a>
</td></tr>
<tr>
<td>M2L</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/York_Mills" title="York Mills">York Mills</a>
</td></tr>
<tr>
<td>M3L</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Downsview" title="Downsview">Downsview West</a>
</td></tr>
<tr>
<td>M4L</td>
<td><a href="/wiki/East_Toronto" title="East Toronto">East Toronto</a></td>
<td>The Beaches West
</td></tr>
<tr>
<td>M4L</td>
<td><a href="/wiki/East_Toronto" title="East Toronto">East Toronto</a></td>
<td><a href="/wiki/India_Bazaar" class="mw-redirect" title="India Bazaar">India Bazaar</a>
</td></tr>
<tr>
<td>M5L</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Commerce_Court" title="Commerce Court">Commerce Court</a>
</td></tr>
<tr>
<td>M5L</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td>Victoria Hotel
</td></tr>
<tr>
<td>M6L</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Maple_Leaf_Park" class="mw-redirect" title="Maple Leaf Park">Maple Leaf Park</a>
</td></tr>
<tr>
<td>M6L</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td>North Park
</td></tr>
<tr>
<td>M6L</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td>Upwood Park
</td></tr>
<tr>
<td>M7L</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8L</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9L</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Humber_Summit" title="Humber Summit">Humber Summit</a>
</td></tr>
<tr>
<td>M1M</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Cliffcrest" title="Cliffcrest">Cliffcrest</a>
</td></tr>
<tr>
<td>M1M</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Cliffside,_Toronto" title="Cliffside, Toronto">Cliffside</a>
</td></tr>
<tr>
<td>M1M</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td>Scarborough Village West
</td></tr>
<tr>
<td>M2M</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Newtonbrook" title="Newtonbrook">Newtonbrook</a>
</td></tr>
<tr>
<td>M2M</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Willowdale,_Toronto" title="Willowdale, Toronto">Willowdale</a>
</td></tr>
<tr>
<td>M3M</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td>Downsview Central
</td></tr>
<tr>
<td>M4M</td>
<td><a href="/wiki/East_Toronto" title="East Toronto">East Toronto</a></td>
<td>Studio District
</td></tr>
<tr>
<td>M5M</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Bedford_Park,_Toronto" title="Bedford Park, Toronto">Bedford Park</a>
</td></tr>
<tr>
<td>M5M</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td>Lawrence Manor East
</td></tr>
<tr>
<td>M6M</td>
<td><a href="/wiki/York" title="York">York</a></td>
<td>Del Ray
</td></tr>
<tr>
<td>M6M</td>
<td><a href="/wiki/York" title="York">York</a></td>
<td><a href="/wiki/Keelesdale" class="mw-redirect" title="Keelesdale">Keelesdale</a>
</td></tr>
<tr>
<td>M6M</td>
<td><a href="/wiki/York" title="York">York</a></td>
<td><a href="/wiki/Mount_Dennis" title="Mount Dennis">Mount Dennis</a>
</td></tr>
<tr>
<td>M6M</td>
<td><a href="/wiki/York" title="York">York</a></td>
<td><a href="/wiki/Silverthorn,_Toronto" title="Silverthorn, Toronto">Silverthorn</a>
</td></tr>
<tr>
<td>M7M</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8M</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9M</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Emery,_Toronto" class="mw-redirect" title="Emery, Toronto">Emery</a>
</td></tr>
<tr>
<td>M9M</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Humberlea" class="mw-redirect" title="Humberlea">Humberlea</a>
</td></tr>
<tr>
<td>M1N</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Birch_Cliff" title="Birch Cliff">Birch Cliff</a>
</td></tr>
<tr>
<td>M1N</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td>Cliffside West
</td></tr>
<tr>
<td>M2N</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td>Willowdale South
</td></tr>
<tr>
<td>M3N</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td>Downsview Northwest
</td></tr>
<tr>
<td>M4N</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td><a href="/wiki/Lawrence_Park,_Toronto" title="Lawrence Park, Toronto">Lawrence Park</a>
</td></tr>
<tr>
<td>M5N</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td>Roselawn
</td></tr>
<tr>
<td>M6N</td>
<td><a href="/wiki/York" title="York">York</a></td>
<td>The Junction North
</td></tr>
<tr>
<td>M6N</td>
<td><a href="/wiki/York" title="York">York</a></td>
<td>Runnymede
</td></tr>
<tr>
<td>M7N</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8N</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9N</td>
<td><a href="/wiki/York" title="York">York</a></td>
<td><a href="/wiki/Weston,_Toronto" title="Weston, Toronto">Weston</a>
</td></tr>
<tr>
<td>M1P</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Dorset_Park" title="Dorset Park">Dorset Park</a>
</td></tr>
<tr>
<td>M1P</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Scarborough_Town_Centre" title="Scarborough Town Centre">Scarborough Town Centre</a>
</td></tr>
<tr>
<td>M1P</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Wexford_Heights" class="mw-redirect" title="Wexford Heights">Wexford Heights</a>
</td></tr>
<tr>
<td>M2P</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td>York Mills West
</td></tr>
<tr>
<td>M3P</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M4P</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td>Davisville North
</td></tr>
<tr>
<td>M5P</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td><a href="/wiki/Forest_Hill_North" class="mw-redirect" title="Forest Hill North">Forest Hill North</a>
</td></tr>
<tr>
<td>M5P</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td>Forest Hill West
</td></tr>
<tr>
<td>M6P</td>
<td><a href="/wiki/West_Toronto" title="West Toronto">West Toronto</a></td>
<td><a href="/wiki/High_Park" title="High Park">High Park</a>
</td></tr>
<tr>
<td>M6P</td>
<td><a href="/wiki/West_Toronto" title="West Toronto">West Toronto</a></td>
<td>The Junction South
</td></tr>
<tr>
<td>M7P</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8P</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9P</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Westmount" class="mw-redirect" title="Westmount">Westmount</a>
</td></tr>
<tr>
<td>M1R</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Maryvale,_Toronto" title="Maryvale, Toronto">Maryvale</a>
</td></tr>
<tr>
<td>M1R</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Wexford" title="Wexford">Wexford</a>
</td></tr>
<tr>
<td>M2R</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Willowdale_West" class="mw-redirect" title="Willowdale West">Willowdale West</a>
</td></tr>
<tr>
<td>M3R</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M4R</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td>North Toronto West
</td></tr>
<tr>
<td>M5R</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td><a href="/wiki/The_Annex" title="The Annex">The Annex</a>
</td></tr>
<tr>
<td>M5R</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td>North Midtown
</td></tr>
<tr>
<td>M5R</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td><a href="/wiki/Yorkville,_Toronto" title="Yorkville, Toronto">Yorkville</a>
</td></tr>
<tr>
<td>M6R</td>
<td><a href="/wiki/West_Toronto" title="West Toronto">West Toronto</a></td>
<td><a href="/wiki/Parkdale,_Toronto" title="Parkdale, Toronto">Parkdale</a>
</td></tr>
<tr>
<td>M6R</td>
<td><a href="/wiki/West_Toronto" title="West Toronto">West Toronto</a></td>
<td><a href="/wiki/Roncesvalles" title="Roncesvalles">Roncesvalles</a>
</td></tr>
<tr>
<td>M7R</td>
<td>Mississauga</td>
<td>Canada Post Gateway Processing Centre
</td></tr>
<tr>
<td>M8R</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9R</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Kingsview_Village" title="Kingsview Village">Kingsview Village</a>
</td></tr>
<tr>
<td>M9R</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Martin Grove Gardens
</td></tr>
<tr>
<td>M9R</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Richview Gardens
</td></tr>
<tr>
<td>M9R</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>St. Phillips
</td></tr>
<tr>
<td>M1S</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Agincourt,_Toronto" title="Agincourt, Toronto">Agincourt</a>
</td></tr>
<tr>
<td>M2S</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3S</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M4S</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td>Davisville
</td></tr>
<tr>
<td>M5S</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td>Harbord
</td></tr>
<tr>
<td>M5S</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/University_of_Toronto" title="University of Toronto">University of Toronto</a>
</td></tr>
<tr>
<td>M6S</td>
<td><a href="/wiki/West_Toronto" title="West Toronto">West Toronto</a></td>
<td><a href="/wiki/Runnymede" title="Runnymede">Runnymede</a>
</td></tr>
<tr>
<td>M6S</td>
<td><a href="/wiki/West_Toronto" title="West Toronto">West Toronto</a></td>
<td><a href="/wiki/Swansea" title="Swansea">Swansea</a>
</td></tr>
<tr>
<td>M7S</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8S</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9S</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M1T</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td>Clarks Corners
</td></tr>
<tr>
<td>M1T</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td>Sullivan
</td></tr>
<tr>
<td>M1T</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Tam_O%27Shanter_%E2%80%93_Sullivan" title="Tam O&#39;Shanter – Sullivan">Tam O'Shanter</a>
</td></tr>
<tr>
<td>M2T</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3T</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M4T</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td><a href="/wiki/Moore_Park,_Toronto" title="Moore Park, Toronto">Moore Park</a>
</td></tr>
<tr>
<td>M4T</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td>Summerhill East
</td></tr>
<tr>
<td>M5T</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Chinatown" title="Chinatown">Chinatown</a>
</td></tr>
<tr>
<td>M5T</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Grange_Park_(Toronto)" title="Grange Park (Toronto)">Grange Park</a>
</td></tr>
<tr>
<td>M5T</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Kensington_Market" title="Kensington Market">Kensington Market</a>
</td></tr>
<tr>
<td>M6T</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M7T</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8T</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9T</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M1V</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Agincourt_North" class="mw-redirect" title="Agincourt North">Agincourt North</a>
</td></tr>
<tr>
<td>M1V</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td>L'Amoreaux East
</td></tr>
<tr>
<td>M1V</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Milliken,_Ontario" title="Milliken, Ontario">Milliken</a>
</td></tr>
<tr>
<td>M1V</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td>Steeles East
</td></tr>
<tr>
<td>M2V</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3V</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M4V</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td><a href="/wiki/Deer_Park,_Toronto" title="Deer Park, Toronto">Deer Park</a>
</td></tr>
<tr>
<td>M4V</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td>Forest Hill SE
</td></tr>
<tr>
<td>M4V</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td><a href="/wiki/Rathnelly" class="mw-redirect" title="Rathnelly">Rathnelly</a>
</td></tr>
<tr>
<td>M4V</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td><a href="/wiki/South_Hill,_Toronto" title="South Hill, Toronto">South Hill</a>
</td></tr>
<tr>
<td>M4V</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td>Summerhill West
</td></tr>
<tr>
<td>M5V</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/CN_Tower" title="CN Tower">CN Tower</a>
</td></tr>
<tr>
<td>M5V</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td>Bathurst Quay
</td></tr>
<tr>
<td>M5V</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td>Island airport
</td></tr>
<tr>
<td>M5V</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td>Harbourfront West
</td></tr>
<tr>
<td>M5V</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/King_and_Spadina" class="mw-redirect" title="King and Spadina">King and Spadina</a>
</td></tr>
<tr>
<td>M5V</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Railway_Lands" title="Railway Lands">Railway Lands</a>
</td></tr>
<tr>
<td>M5V</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/South_Niagara" class="mw-redirect" title="South Niagara">South Niagara</a>
</td></tr>
<tr>
<td>M6V</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M7V</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8V</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Humber Bay Shores
</td></tr>
<tr>
<td>M8V</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Mimico South
</td></tr>
<tr>
<td>M8V</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/New_Toronto" title="New Toronto">New Toronto</a>
</td></tr>
<tr>
<td>M9V</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Albion Gardens
</td></tr>
<tr>
<td>M9V</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Beaumond_Heights" class="mw-redirect" title="Beaumond Heights">Beaumond Heights</a>
</td></tr>
<tr>
<td>M9V</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Humbergate
</td></tr>
<tr>
<td>M9V</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Mount_Olive-Silverstone-Jamestown" class="mw-redirect" title="Mount Olive-Silverstone-Jamestown">Jamestown</a>
</td></tr>
<tr>
<td>M9V</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Mount_Olive-Silverstone-Jamestown" class="mw-redirect" title="Mount Olive-Silverstone-Jamestown">Mount Olive</a>
</td></tr>
<tr>
<td>M9V</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Silverstone" title="Silverstone">Silverstone</a>
</td></tr>
<tr>
<td>M9V</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/South_Steeles" class="mw-redirect" title="South Steeles">South Steeles</a>
</td></tr>
<tr>
<td>M9V</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Thistletown" title="Thistletown">Thistletown</a>
</td></tr>
<tr>
<td>M1W</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td>L'Amoreaux West
</td></tr>
<tr>
<td>M1W</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Steeles_West" class="mw-redirect" title="Steeles West">Steeles West</a>
</td></tr>
<tr>
<td>M2W</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3W</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M4W</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Rosedale,_Toronto" title="Rosedale, Toronto">Rosedale</a>
</td></tr>
<tr>
<td>M5W</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td>Stn A PO Boxes 25 The Esplanade
</td></tr>
<tr>
<td>M6W</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M7W</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8W</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Alderwood,_Toronto" title="Alderwood, Toronto">Alderwood</a>
</td></tr>
<tr>
<td>M8W</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Long_Branch,_Toronto" title="Long Branch, Toronto">Long Branch</a>
</td></tr>
<tr>
<td>M9W</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Northwest" class="mw-redirect" title="Northwest">Northwest</a>
</td></tr>
<tr>
<td>M1X</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Upper_Rouge" class="mw-redirect" title="Upper Rouge">Upper Rouge</a>
</td></tr>
<tr>
<td>M2X</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3X</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M4X</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Cabbagetown,_Toronto" title="Cabbagetown, Toronto">Cabbagetown</a>
</td></tr>
<tr>
<td>M4X</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/St._James_Town" title="St. James Town">St. James Town</a>
</td></tr>
<tr>
<td>M5X</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/First_Canadian_Place" title="First Canadian Place">First Canadian Place</a>
</td></tr>
<tr>
<td>M5X</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Underground_city" title="Underground city">Underground city</a>
</td></tr>
<tr>
<td>M6X</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M7X</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8X</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/The_Kingsway" title="The Kingsway">The Kingsway</a>
</td></tr>
<tr>
<td>M8X</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Montgomery Road
</td></tr>
<tr>
<td>M8X</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Old Mill North
</td></tr>
<tr>
<td>M9X</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M1Y</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2Y</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3Y</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M4Y</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Church_and_Wellesley" title="Church and Wellesley">Church and Wellesley</a>
</td></tr>
<tr>
<td>M5Y</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M6Y</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M7Y</td>
<td><a href="/wiki/East_Toronto" title="East Toronto">East Toronto</a></td>
<td>Business Reply Mail Processing Centre 969 Eastern
</td></tr>
<tr>
<td>M8Y</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Humber_Bay" title="Humber Bay">Humber Bay</a>
</td></tr>
<tr>
<td>M8Y</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Kingsmills_Park" title="Kingsmills Park">King's Mill Park</a>
</td></tr>
<tr>
<td>M8Y</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Kingsway Park South East
</td></tr>
<tr>
<td>M8Y</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Mimico" title="Mimico">Mimico NE</a>
</td></tr>
<tr>
<td>M8Y</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Old_Mill,_Toronto" title="Old Mill, Toronto">Old Mill South</a>
</td></tr>
<tr>
<td>M8Y</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/The_Queensway" title="The Queensway">The Queensway East</a>
</td></tr>
<tr>
<td>M8Y</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Fairmont_Royal_York_Hotel" class="mw-redirect" title="Fairmont Royal York Hotel">Royal York South East</a>
</td></tr>
<tr>
<td>M8Y</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Sunnylea" class="mw-redirect" title="Sunnylea">Sunnylea</a>
</td></tr>
<tr>
<td>M9Y</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M1Z</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2Z</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3Z</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M4Z</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M5Z</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M6Z</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M7Z</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8Z</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Kingsway Park South West
</td></tr>
<tr>
<td>M8Z</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Mimico" title="Mimico">Mimico NW</a>
</td></tr>
<tr>
<td>M8Z</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/The_Queensway" title="The Queensway">The Queensway West</a>
</td></tr>
<tr>
<td>M8Z</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Royal York South West
</td></tr>
<tr>
<td>M8Z</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Bloor" title="Bloor">South of Bloor</a>
</td></tr>
<tr>
<td>M9Z</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
</tbody></table>
"""

In [3]:
# Transforming Wikitable into Pandas Dataframe
from IPython.display import display_html
display_html(html_string, raw=True)

neighbourhoods = pd.read_html(html_string, header = 0)
neighbourhoods = neighbourhoods[0]
neighbourhoods

Postcode,Borough,Neighbourhood
M1A,Not assigned,Not assigned
M2A,Not assigned,Not assigned
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M5A,Downtown Toronto,Regent Park
M6A,North York,Lawrence Heights
M6A,North York,Lawrence Manor
M7A,Queen's Park,Not assigned
M8A,Not assigned,Not assigned


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [4]:
# Dropping 'Not assigned' Boroughs
neighbourhoods = neighbourhoods[neighbourhoods.Borough != 'Not assigned']
neighbourhoods

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [5]:
# Grouping based on Borough
neighbourhoods = neighbourhoods.groupby(['Postcode','Borough'], sort=False).Neighbourhood.apply(', '.join).reset_index()
neighbourhoods

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [6]:
# Replacing 'Not assigned' in the 'Neighbourhood' column
rep = neighbourhoods['Borough']
neighbourhoods['Neighbourhood'] = rep.where(neighbourhoods['Neighbourhood'] == 'Not assigned', other = neighbourhoods['Neighbourhood'])
neighbourhoods

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [7]:
# Dimensions of the Dataframe
neighbourhoods.shape

(103, 3)

### Importing Geographical Coordinates of the Neighborhoods

In [8]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Joining Neighbourhoods and Geographical Coordinates

In [9]:
# Renaming columns to join
neighbourhoods.rename(columns={'Postcode':'PostalCode'}, inplace=True)
df_data_1.rename(columns={'Postal Code':'PostalCode'}, inplace=True)

# Joining
neighbourhoods = neighbourhoods.join(df_data_1.set_index('PostalCode'), on='PostalCode')
neighbourhoods

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


### Creating a Map of Toronto with Neighborhoods

In [10]:
# Latitude and longitude of Toronto
latitude = 43.6532
longitude = -79.3832

# Creating a map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# Adding markers to map
for lat, lng, borough, neighbourhood in zip(neighbourhoods['Latitude'], neighbourhoods['Longitude'], neighbourhoods['Borough'], neighbourhoods['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Foursquare Credentials

In [11]:
# The code was removed by Watson Studio for sharing.

### Getting Nearby Venues for All Neighbourhoods

In [12]:
# Defining 'getNearbyVenues' function

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
# Getting Toronto venues and putting them into Pandas dataframe

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

toronto_venues = getNearbyVenues(names=neighbourhoods['Neighbourhood'],
                                   latitudes=neighbourhoods['Latitude'],
                                   longitudes=neighbourhoods['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront, Regent Park
Lawrence Heights, Lawrence Manor
Queen's Park
Islington Avenue
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The D

In [14]:
# Size of the Toronto venue dataframe
print(toronto_venues.shape)
toronto_venues[0:20]

(2238, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
5,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
6,Victoria Village,43.725882,-79.315572,Eglinton Ave E & Sloane Ave/Bermondsey Rd,43.726086,-79.313620,Intersection
7,"Harbourfront, Regent Park",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
8,"Harbourfront, Regent Park",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
9,"Harbourfront, Regent Park",43.654260,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center


In [15]:
# Counting numbers of venues
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",9,9,9,9,9,9
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Downsview North, Wilson Heights",17,17,17,17,17,17
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
Berczy Park,56,56,56,56,56,56


In [16]:
# Counting unique categories of these venues
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 274 uniques categories.


### Analyzing Each Toronto Neighbourhood

In [17]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [18]:
toronto_onehot.shape

(2238, 275)

In [19]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.00

In [20]:
toronto_grouped.shape

(100, 275)

In [21]:
# Finding top 5 venue categories in each neighbourhood
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2           Steakhouse  0.04
3  American Restaurant  0.04
4      Thai Restaurant  0.04


----Agincourt----
               venue  freq
0     Breakfast Spot  0.25
1             Lounge  0.25
2       Skating Rink  0.25
3     Clothing Store  0.25
4  Accessories Store  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
               venue  freq
0         Playground   0.5
1               Park   0.5
2  Accessories Store   0.0
3  Mobile Phone Shop   0.0
4      Moving Target   0.0


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0        Grocery Store  0.22
1             Pharmacy  0.11
2           Beer Store  0.11
3  Fried Chicken Joint  0.11
4          Coffee Shop  0.11


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.22
1      

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
# Finding 10 most common venues in each neighbourhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,American Restaurant,Clothing Store,Gym,Hotel,Bakery,Bar
1,Agincourt,Lounge,Breakfast Spot,Skating Rink,Clothing Store,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pharmacy,Coffee Shop,Beer Store,Sandwich Place,Fried Chicken Joint,Fast Food Restaurant,Pizza Place,Construction & Landscaping,Concert Hall
4,"Alderwood, Long Branch",Pizza Place,Pub,Gym,Sandwich Place,Pharmacy,Coffee Shop,Skating Rink,Pool,Yoga Studio,Diner


### Clustering Toronto Neighbourhoods

In [24]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 1, 0, 2, 2, 2, 2, 2, 2], dtype=int32)

In [25]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'ClusterLabels', kmeans.labels_)

toronto_merged = neighbourhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighbourhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged[0:10] # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Fast Food Restaurant,Park,Food & Drink Shop,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Empanada Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,2.0,Intersection,Hockey Arena,Coffee Shop,Portuguese Restaurant,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,2.0,Coffee Shop,Park,Bakery,Café,Theater,Breakfast Spot,Pub,Mexican Restaurant,Brewery,Shoe Store
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,2.0,Clothing Store,Accessories Store,Coffee Shop,Event Space,Sporting Goods Shop,Miscellaneous Shop,Furniture / Home Store,Gift Shop,Boutique,Women's Store
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,2.0,Coffee Shop,Sushi Restaurant,Gym,Japanese Restaurant,Diner,Spa,Portuguese Restaurant,Bar,Café,Italian Restaurant
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0.0,Fast Food Restaurant,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant
7,M3B,North York,Don Mills North,43.745906,-79.352188,2.0,Café,Baseball Field,Caribbean Restaurant,Gym / Fitness Center,Japanese Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937,0.0,Fast Food Restaurant,Pizza Place,Pharmacy,Gastropub,Bank,Rock Climbing Spot,Athletics & Sports,Intersection,Breakfast Spot,Pet Store
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,2.0,Clothing Store,Coffee Shop,Café,Middle Eastern Restaurant,Cosmetics Shop,Diner,Restaurant,Tea Room,Ramen Restaurant,Thai Restaurant


In [26]:
# Checking data types of all columns
toronto_merged.dtypes

PostalCode                 object
Borough                    object
Neighbourhood              object
Latitude                  float64
Longitude                 float64
ClusterLabels             float64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

In [27]:
# Dropping rows that have 'NaN' under 'Cluster Labels'
toronto_merged = toronto_merged.drop([5,52,95])

# Changing the datatype of 'Cluster labels' to int64
toronto_merged['ClusterLabels'] = toronto_merged['ClusterLabels'].astype(int)
toronto_merged

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1,Fast Food Restaurant,Park,Food & Drink Shop,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Empanada Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,2,Intersection,Hockey Arena,Coffee Shop,Portuguese Restaurant,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,2,Coffee Shop,Park,Bakery,Café,Theater,Breakfast Spot,Pub,Mexican Restaurant,Brewery,Shoe Store
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,2,Clothing Store,Accessories Store,Coffee Shop,Event Space,Sporting Goods Shop,Miscellaneous Shop,Furniture / Home Store,Gift Shop,Boutique,Women's Store
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,2,Coffee Shop,Sushi Restaurant,Gym,Japanese Restaurant,Diner,Spa,Portuguese Restaurant,Bar,Café,Italian Restaurant
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0,Fast Food Restaurant,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant
7,M3B,North York,Don Mills North,43.745906,-79.352188,2,Café,Baseball Field,Caribbean Restaurant,Gym / Fitness Center,Japanese Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937,0,Fast Food Restaurant,Pizza Place,Pharmacy,Gastropub,Bank,Rock Climbing Spot,Athletics & Sports,Intersection,Breakfast Spot,Pet Store
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,2,Clothing Store,Coffee Shop,Café,Middle Eastern Restaurant,Cosmetics Shop,Diner,Restaurant,Tea Room,Ramen Restaurant,Thai Restaurant
10,M6B,North York,Glencairn,43.709577,-79.445073,1,Pizza Place,Park,Japanese Restaurant,Metro Station,Pub,Yoga Studio,Drugstore,Discount Store,Dog Run,Doner Restaurant


In [28]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examining Clusters

In [29]:
# Cluster 1
toronto_merged.loc[toronto_merged['ClusterLabels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,0,Fast Food Restaurant,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant
8,East York,0,Fast Food Restaurant,Pizza Place,Pharmacy,Gastropub,Bank,Rock Climbing Spot,Athletics & Sports,Intersection,Breakfast Spot,Pet Store
70,Etobicoke,0,Middle Eastern Restaurant,Pizza Place,Sandwich Place,Coffee Shop,Chinese Restaurant,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop
72,North York,0,Pharmacy,Pizza Place,Butcher,Grocery Store,Coffee Shop,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop
82,Scarborough,0,Pizza Place,Pharmacy,Noodle House,Fried Chicken Joint,Italian Restaurant,Thai Restaurant,Chinese Restaurant,Fast Food Restaurant,Cosmetics Shop,Comfort Food Restaurant
89,Etobicoke,0,Grocery Store,Pharmacy,Coffee Shop,Beer Store,Sandwich Place,Fried Chicken Joint,Fast Food Restaurant,Pizza Place,Construction & Landscaping,Concert Hall
90,Scarborough,0,Fast Food Restaurant,Chinese Restaurant,Japanese Restaurant,Pizza Place,Coffee Shop,Sandwich Place,Electronics Store,Breakfast Spot,Pharmacy,Grocery Store


In [30]:
# Cluster 2
toronto_merged.loc[toronto_merged['ClusterLabels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1,Fast Food Restaurant,Park,Food & Drink Shop,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Empanada Restaurant
10,North York,1,Pizza Place,Park,Japanese Restaurant,Metro Station,Pub,Yoga Studio,Drugstore,Discount Store,Dog Run,Doner Restaurant
21,York,1,Park,Pharmacy,Fast Food Restaurant,Market,Women's Store,Comic Shop,Discount Store,Farmers Market,Falafel Restaurant,Event Space
35,East York,1,Park,Convenience Store,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
40,North York,1,Park,Airport,Other Repair Shop,Bus Stop,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
49,North York,1,Basketball Court,Park,Construction & Landscaping,Bakery,Empanada Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
61,Central Toronto,1,Dim Sum Restaurant,Park,Bus Line,Swim School,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
64,York,1,Park,Convenience Store,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
66,North York,1,Park,Bank,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
68,Central Toronto,1,Trail,Park,Sushi Restaurant,Jewelry Store,Yoga Studio,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant


In [31]:
# Cluster 3
toronto_merged.loc[toronto_merged['ClusterLabels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,2,Intersection,Hockey Arena,Coffee Shop,Portuguese Restaurant,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,Downtown Toronto,2,Coffee Shop,Park,Bakery,Café,Theater,Breakfast Spot,Pub,Mexican Restaurant,Brewery,Shoe Store
3,North York,2,Clothing Store,Accessories Store,Coffee Shop,Event Space,Sporting Goods Shop,Miscellaneous Shop,Furniture / Home Store,Gift Shop,Boutique,Women's Store
4,Queen's Park,2,Coffee Shop,Sushi Restaurant,Gym,Japanese Restaurant,Diner,Spa,Portuguese Restaurant,Bar,Café,Italian Restaurant
7,North York,2,Café,Baseball Field,Caribbean Restaurant,Gym / Fitness Center,Japanese Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
9,Downtown Toronto,2,Clothing Store,Coffee Shop,Café,Middle Eastern Restaurant,Cosmetics Shop,Diner,Restaurant,Tea Room,Ramen Restaurant,Thai Restaurant
11,Etobicoke,2,Filipino Restaurant,Bank,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,Discount Store
12,Scarborough,2,Bar,Moving Target,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
13,North York,2,Gym,Asian Restaurant,Beer Store,Coffee Shop,Italian Restaurant,General Entertainment,Japanese Restaurant,Dim Sum Restaurant,Chinese Restaurant,Sandwich Place
14,East York,2,Pharmacy,Intersection,Cosmetics Shop,Curling Ice,Dance Studio,Bus Stop,Skating Rink,Beer Store,Park,Video Store


In [32]:
# Cluster 4
toronto_merged.loc[toronto_merged['ClusterLabels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,North York,3,Baseball Field,Food Truck,Yoga Studio,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
57,North York,3,Baseball Field,Construction & Landscaping,Yoga Studio,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
101,Etobicoke,3,Baseball Field,Yoga Studio,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Discount Store


In [33]:
# Cluster 5
toronto_merged.loc[toronto_merged['ClusterLabels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,North York,4,Cafeteria,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Yoga Studio,Filipino Restaurant
